# Fix xr-unreadable GOCCP files
I will call command-line netcdf operations through python to remove the extraneous 'time' variable.

In [1]:
import sys
# Add common resources folder to path
sys.path.append('/glade/u/home/jonahshaw/Scripts/git_repos/CESM2_analysis')
sys.path.append('/glade/u/home/jonahshaw/Scripts/git_repos/CESM2_analysis/Common/')
# sys.path.append("/home/jonahks/git_repos/netcdf_analysis/Common/")

from imports import (
    pd, np, xr, mpl, plt, sns, os, 
    datetime, sys, crt, gridspec,
    ccrs, metrics, Iterable, xe,
    Path, run
    )

from functions import (
    masked_average, add_weights, sp_map,
    season_mean, get_dpm, leap_year, share_ylims,
    to_png
    )

from interp_functions import *

from cloud_metric import Cloud_Metric
from collections import deque
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ls

CESM2_COSP/         download_goccp.ipynb*   satcomp_metricdev.ipynb*
Common/             fix_1x1goccp2006.ipynb  slurm-6501063.out
COSP_icebug/        Interpolation/          slurm-6501064.out
daskhelp.ipynb      LICENSE
dask-worker-space/  README.md*


In [3]:
obs_dir = '/glade/u/home/jonahshaw/w/obs/'

In [7]:
cd '/glade/u/home/jonahshaw/w/obs/CALIPSO/GOCCP/2Ddata/'

/glade/work/jonahshaw/obs/CALIPSO/GOCCP/2Ddata


In [9]:
ls

1.25x0.9_from_1x1_python/  1x1testing/     grid_2x2_L40/
1.25x0.9_interpolation/    grid_1x1_2006/  remapweights.nc
1.25x0.9_python_interp/    grid_1x1_L40/


In [11]:
og_dir = '/glade/u/home/jonahshaw/w/obs/CALIPSO/GOCCP/2Ddata/grid_1x1_2006/'
os.listdir(og_dir)

['MapLowMidHigh330m_200610_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh330m_200607_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh_Phase330m_200608_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh_Phase330m_200607_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh_Phase330m_200611_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh_Phase330m_200612_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh330m_200612_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh330m_200611_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh_Phase330m_200609_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh330m_200609_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh330m_200608_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh330m_200606_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh_Phase330m_200610_avg_CFMIP1_sat_3.1.2.nc',
 'MapLowMidHigh_Phase330m_200606_avg_CFMIP1_sat_3.1.2.nc']

In [10]:
f'ncks -C -x -v time MapLowMidHigh330m_200606_avg_CFMIP1_sat_3.1.2.nc timeless.nc'

'ncks -C -x -v time MapLowMidHigh330m_200606_avg_CFMIP1_sat_3.1.2.nc timeless.nc'

Value 127 is returned by /bin/sh when the given command is not found within your PATH system variable and it is not a built-in shell command.

## This kept not working for an unclear reason, so I pasted it into a standard python file and executed it via the terminal :(

In [27]:
og_dir = '/glade/u/home/jonahshaw/w/obs/CALIPSO/GOCCP/2Ddata/grid_1x1_2006/'
out_dir = '/glade/u/home/jonahshaw/w/obs/CALIPSO/GOCCP/2Ddata/grid_1x1_2006fix/'

commands = [f'module load nco']
for file in os.listdir(og_dir): # iterate over years (2009-2013)
    _prepath = '%s%s' % (og_dir, file)
    _outpath = '%s%s' % (out_dir, file)
    
    comm = f'ncks -C -x -v time {_prepath} {_outpath}'
    commands.append(comm)
    
for com in commands:
    print(com)
#     run(com)
    out = run(com, cwd='/glade/u/home/jonahshaw/w/obs/', shell=True)
    print(out)


#     _files = os.listdir(_prepath)

#     print(os.path.exists(_prepath))

#     _phasefiles = [_prepath + x for x in _files if 'Phase' in x]
#     _cloudfiles = [_prepath + x for x in _files if 'MapLowMidHigh330m' in x]

#     phase = phase + _phasefiles
#     cloud = cloud + _cloudfiles

module load nco
CompletedProcess(args='module load nco', returncode=0)
ncks -C -x -v time /glade/u/home/jonahshaw/w/obs/CALIPSO/GOCCP/2Ddata/grid_1x1_2006/MapLowMidHigh330m_200610_avg_CFMIP1_sat_3.1.2.nc /glade/u/home/jonahshaw/w/obs/CALIPSO/GOCCP/2Ddata/grid_1x1_2006fix/MapLowMidHigh330m_200610_avg_CFMIP1_sat_3.1.2.nc
CompletedProcess(args='ncks -C -x -v time /glade/u/home/jonahshaw/w/obs/CALIPSO/GOCCP/2Ddata/grid_1x1_2006/MapLowMidHigh330m_200610_avg_CFMIP1_sat_3.1.2.nc /glade/u/home/jonahshaw/w/obs/CALIPSO/GOCCP/2Ddata/grid_1x1_2006fix/MapLowMidHigh330m_200610_avg_CFMIP1_sat_3.1.2.nc', returncode=127)
ncks -C -x -v time /glade/u/home/jonahshaw/w/obs/CALIPSO/GOCCP/2Ddata/grid_1x1_2006/MapLowMidHigh330m_200607_avg_CFMIP1_sat_3.1.2.nc /glade/u/home/jonahshaw/w/obs/CALIPSO/GOCCP/2Ddata/grid_1x1_2006fix/MapLowMidHigh330m_200607_avg_CFMIP1_sat_3.1.2.nc
CompletedProcess(args='ncks -C -x -v time /glade/u/home/jonahshaw/w/obs/CALIPSO/GOCCP/2Ddata/grid_1x1_2006/MapLowMidHigh330m_200607_avg_CF